# Summarize epitopes by antigen

Given a list of pubmed IDs, use the IQ-API to pull in all antigen and epitope information
and summarize as follows:

* antigen_id, name, etc.
* number of reference that report positive epitopes derived from that antigen
* total num_peptides_positive derived from each antigen (each peptide is counted only once even if it appears in multiple references)

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import io
import time
import requests
import pandas as pd

In [2]:
# Read in the pubmed IDs from a file:
with open('pubmed_ids.txt') as f:
    pubmed_ids = [i.rstrip() for i in f.readlines()]
    pubmed_id_string = ','.join(pubmed_ids)

Let's define a function for running a query and returning a dataframe of results.  Since there may be more than 10K records, we will need to fetch 1 page at a time and continue to append them onto our dataframe. 

**NOTE:** When paging through results by using an 'offset', it is critical to add an 'order' parameter to ensure that the pages are consistent between queries.

To be good citizens, we also add a short 'sleep' in between calls to the API.

In [3]:
def iq_query(endpoint, query_params, base_uri='https://query-api.iedb.org/'):

    url = os.path.join(base_uri, endpoint)
    df = pd.DataFrame()
    
    # set the offset to 0
    query_params['offset'] = 0
    
    # loop through the pages of results
    # API only allows pulling 10,000 entries at a time
    while(True):
        print('Fetching offset: %i' % query_params['offset'])
        r = requests.get(url, params=query_params, headers={'accept': 'text/csv', 'Prefer': 'count=exact'})
        try:
            df = pd.concat([df, pd.read_csv(io.StringIO(r.content.decode('utf-8')))])
            query_params['offset'] += 10000
        except pd.errors.EmptyDataError:
            break
        
        # sleep for 1 second between calls so as not to overload the server
        time.sleep(1)
    
    return df 

### Map the pubmed IDs to reference IDs

Let's set the query string parameters

In [4]:
query_params = {'pubmed_id': 'in.(%s)' % pubmed_id_string,
                'order': 'pubmed_id',
                'select': 'pubmed_id,reference_id'}

And pull the data and create a reference ID string to incorporate into downstream queries:

In [5]:
ref2pmid = iq_query('reference_search', query_params)
ref_id_string = ','.join(list(ref2pmid['reference_id'].astype(str)))

Fetching offset: 0
Fetching offset: 10000


### Fetch the positive T & B cell epitopes
First, set the query parameters:

In [6]:
query_params = {'reference_id': 'in.(%s)' % ref_id_string,
                'qualitative_measure': 'neq.Negative',
                'order': 'pubmed_id',
                'select': 'reference_id,parent_source_antigen_iri,structure_id,structure_description,curated_source_antigen'}

Now run the query against tcell_search

In [7]:
tcell_epitopes = iq_query('tcell_search', query_params)

Fetching offset: 0
Fetching offset: 10000


And then B cell search:

In [8]:
bcell_epitopes = iq_query('bcell_search', query_params)

Fetching offset: 0
Fetching offset: 10000


**NOTE** There are certain cases when the source antigen is unknown/null.  We'll quantify
those here, but will ignore those epitopes for the remainder of this analysis.

### Initial summary

Here we summarize the number of T & B cell epitopes retrieved and the number where the source antigen (parent protein) is not null.  We move forward by combining these data.

In [9]:
# "~" operator is NOT; remove epitopes without a source antigen
tcell_epitopes = tcell_epitopes[~tcell_epitopes['parent_source_antigen_iri'].isna()]
bcell_epitopes = bcell_epitopes[~bcell_epitopes['parent_source_antigen_iri'].isna()]

In both cases, more so in B cell, there are many records that are not mapped to a parent protein.  We'll investigate down below.

Let's combine all epitopes into one dataframe.

In [10]:
all_epitopes = pd.concat([tcell_epitopes, bcell_epitopes])

### References and peptides per parent antigen

Let's get the number of references and peptides (including discontinuous sequences) per parent antigen for each parent antigen first.

In [11]:
ref_data  = []
peps_data = []
for i, row in all_epitopes.groupby('parent_source_antigen_iri'):
    ref_data.append([i, len(row['reference_id'].unique())])
    peps_data.append([i, len(row['structure_id'].unique())])

refs_per_parent_antigen = pd.DataFrame(data=ref_data, columns=['parent_source_antigen_iri', 'num_references'])
peps_per_parent_antigen = pd.DataFrame(data=peps_data, columns=['parent_source_antigen_iri', 'num_peptides'])


In [17]:
refs_per_parent_antigen

,parent_source_antigen_iri,num_references
0,UNIPROT:A2JGV3,1
1,UNIPROT:A5PLL7,3
2,UNIPROT:I3L0A0,1
3,UNIPROT:O15438,2
4,UNIPROT:O15466,1
5,UNIPROT:O42043,1
6,UNIPROT:O75638,1
7,UNIPROT:P00533,2
8,UNIPROT:P02782,1
9,UNIPROT:P06239,3


### Source antigen (parent protein) data

Before we put everything together, we need to pull all of the source antigen information.  Since the parent_source_antigen_iri is not yet in the parent_proteins table, we first pull ALL of the parent proteins
and create the parent_source_antigen_uri later.

In [12]:
query_params = {'order': 'accession',
                'select': 'database,accession,name,title,proteome_label'}

Now run the query against parent_proteins to pull everything out

In [13]:
parent_proteins = iq_query('parent_proteins', query_params)

Fetching offset: 0
Fetching offset: 10000
Fetching offset: 20000
Fetching offset: 30000
Fetching offset: 40000
Fetching offset: 50000
Fetching offset: 60000
Fetching offset: 70000
Fetching offset: 80000
Fetching offset: 90000
Fetching offset: 100000


In [14]:
# let's add a psuedo parent_source_antigen_iri
parent_proteins['parent_source_antigen_iri'] = parent_proteins['database'].str.upper() + ':' + parent_proteins['accession'].str[:]

### Final summary by parent protein

Here, we combine the data into the table of interest

In [15]:
final_summary_by_parent = pd.merge(refs_per_parent_antigen, peps_per_parent_antigen, how='left', on='parent_source_antigen_iri')
final_summary_by_parent = pd.merge(final_summary_by_parent, parent_proteins, how='left', on='parent_source_antigen_iri')

# rename some columns
final_summary_by_parent.rename(columns={'parent_source_antigen_iri': 'antigen_id',
                                        'title': 'antigen_title'}, inplace=True)

# sort by number of references and number of peptides
final_summary_by_parent = final_summary_by_parent.sort_values('num_references', ascending=False)
final_summary_by_parent = final_summary_by_parent.sort_values('num_peptides', ascending=False)

final_summary_by_parent

,antigen_id,num_references,num_peptides,database,accession,name,antigen_title,proteome_label
25,UNIPROT:Q15020,4,8,UniProt,Q15020,sp|Q15020|SART3_HUMAN,Squamous cell carcinoma antigen recognized by ...,Homo sapiens (Human)
13,UNIPROT:P12272,2,8,UniProt,P12272,sp|P12272|PTHR_HUMAN,Parathyroid hormone-related protein,Homo sapiens (Human)
9,UNIPROT:P06239,3,7,UniProt,P06239,sp|P06239|LCK_HUMAN,Tyrosine-protein kinase Lck,Homo sapiens (Human)
17,UNIPROT:P22732,1,7,UniProt,P22732,sp|P22732|GTR5_HUMAN,"Solute carrier family 2, facilitated glucose t...",Homo sapiens (Human)
21,UNIPROT:P63145,1,6,UniProt,P63145,sp|P63145|GAK24_HUMAN,Endogenous retrovirus group K member 24 Gag po...,Homo sapiens (Human)
20,UNIPROT:P62684,1,6,UniProt,P62684,sp|P62684|GA113_HUMAN,Endogenous retrovirus group K member 113 Gag p...,Homo sapiens (Human)
10,UNIPROT:P07288,3,6,UniProt,P07288,sp|P07288|KLK3_HUMAN,Prostate-specific antigen,Homo sapiens (Human)
19,UNIPROT:P60484,1,5,UniProt,P60484,sp|P60484|PTEN_HUMAN,"Phosphatidylinositol 3,4,5-trisphosphate 3-pho...",Homo sapiens (Human)
23,UNIPROT:Q04609,2,4,UniProt,Q04609,sp|Q04609|FOLH1_HUMAN,Glutamate carboxypeptidase 2,Homo sapiens (Human)
8,UNIPROT:P02782,1,4,UniProt,P02782,sp|P02782|PSC1_RAT,Prostatic steroid-binding protein C1,Rattus norvegicus (Rat) (Strain: Brown Norway)


As we can see by the row with an empty antigen_id & antigen_title, there are a good number of proteins that are not mapped to a parent protein.  So let's have a closer look at those, specifically:

In [16]:
all_epitopes['curated_antigen_accession'] = all_epitopes['curated_source_antigen'].str.split(',').str[0].str[1:]
all_epitopes[all_epitopes['parent_source_antigen_iri'].isna()]

,reference_id,parent_source_antigen_iri,structure_id,structure_description,curated_source_antigen,curated_antigen_accession


Interesting!  None of them have curated source antigens, so we should investigate why and ignore them if there is a good explanation.